In [121]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from datetime import date, datetime

DB_NAME = "publictransportation"
DB_USER = "postgres"
DB_PASS = "hellosql"
DB_HOST = "localhost"
DB_PORT = "5432"

In [142]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")
conn = engine.connect()

sql_query = """
CREATE TABLE if not exists monthly_totals
(id serial,
month varchar,
year int,
city varchar,
transportation_type varchar,
ridership_count int
);
"""

conn.execute(text(sql_query))
conn.commit()

AttributeError: 'Connection' object has no attribute 'commit'

#### Successfully able to read in the two dataset files into DataFrames

In [54]:
chicago_df = pd.read_csv('./data/CTA_-_Ridership_-_Daily_Boarding_Totals_20240307.csv')
chicago_df.head()

,service_date,day_type,bus,rail_boardings,total_rides
0,01/01/2001,U,297192,126455,423647
1,01/02/2001,W,780827,501952,1282779
2,01/03/2001,W,824923,536432,1361355
3,01/04/2001,W,870021,550011,1420032
4,01/05/2001,W,890426,557917,1448343


In [94]:
newyork_df = pd.read_csv('./data/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv')
newyork_df.head()

,Month,Agency,Ridership
0,01/01/2008,MNR,6453734
1,02/01/2008,MNR,6144377
2,03/01/2008,MNR,6602280
3,04/01/2008,MNR,6787480
4,04/01/2008,B&T,24900328


#### Ensured each column is appropriately typed

In [56]:
chicago_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8401 entries, 0 to 8400
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   service_date    8401 non-null   object
 1   day_type        8401 non-null   object
 2   bus             8401 non-null   int64 
 3   rail_boardings  8401 non-null   int64 
 4   total_rides     8401 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 328.3+ KB


In [57]:
newyork_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Month      648 non-null    object
 1   Agency     648 non-null    object
 2   Ridership  648 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 15.3+ KB


In [58]:
chicago_df['service_date'] = pd.to_datetime(chicago_df['service_date'])
chicago_df.head()

,service_date,day_type,bus,rail_boardings,total_rides
0,2001-01-01,U,297192,126455,423647
1,2001-01-02,W,780827,501952,1282779
2,2001-01-03,W,824923,536432,1361355
3,2001-01-04,W,870021,550011,1420032
4,2001-01-05,W,890426,557917,1448343


In [95]:
newyork_df['Month'] = pd.to_datetime(newyork_df['Month'])
newyork_df.head()

,Month,Agency,Ridership
0,2008-01-01,MNR,6453734
1,2008-02-01,MNR,6144377
2,2008-03-01,MNR,6602280
3,2008-04-01,MNR,6787480
4,2008-04-01,B&T,24900328


#### Correct use of pandas to transform any necessary columns

In [70]:
#### CHICAGO

In [60]:
chicago_df = chicago_df.drop(columns = ['day_type', 'total_rides'])
chicago_df.head()

,service_date,bus,rail_boardings
0,2001-01-01,297192,126455
1,2001-01-02,780827,501952
2,2001-01-03,824923,536432
3,2001-01-04,870021,550011
4,2001-01-05,890426,557917


In [61]:
new_chicago_df = pd.melt(chicago_df, id_vars = ['service_date'], value_vars = ['bus', 'rail_boardings'], var_name = 'transportation_type', value_name = 'ridership_count')
new_chicago_df.head()

,service_date,transportation_type,ridership_count
0,2001-01-01,bus,297192
1,2001-01-02,bus,780827
2,2001-01-03,bus,824923
3,2001-01-04,bus,870021
4,2001-01-05,bus,890426


In [63]:
new_chicago_df['transportation_type'].mask(new_chicago_df['transportation_type'] == 'rail_boardings', 'metro', inplace = True)

In [64]:
new_chicago_df['city'] = 'Chicago'
new_chicago_df.head()

,service_date,transportation_type,ridership_count,city
0,2001-01-01,bus,297192,Chicago
1,2001-01-02,bus,780827,Chicago
2,2001-01-03,bus,824923,Chicago
3,2001-01-04,bus,870021,Chicago
4,2001-01-05,bus,890426,Chicago


In [65]:
def find_month(date):
    if date.month == 1:
        return 'January'
    elif date.month == 2:
        return 'February'
    elif date.month == 3:
        return 'March'
    elif date.month == 4:
        return 'April'
    elif date.month == 5:
        return 'May'
    elif date.month == 6:
        return 'June'
    elif date.month == 7:
        return 'July'
    elif date.month == 8:
        return 'August'
    elif date.month == 9:
        return 'September'
    elif date.month == 10:
        return 'October'
    elif date.month == 11:
        return 'November'
    elif date.month == 12:
        return 'December'
    else:
        return 'Invalid month'

new_chicago_df['month'] = new_chicago_df['service_date'].apply(find_month)
new_chicago_df.head()

,service_date,transportation_type,ridership_count,city,month
0,2001-01-01,bus,297192,Chicago,January
1,2001-01-02,bus,780827,Chicago,January
2,2001-01-03,bus,824923,Chicago,January
3,2001-01-04,bus,870021,Chicago,January
4,2001-01-05,bus,890426,Chicago,January


In [66]:
new_chicago_df['year'] = new_chicago_df['service_date'].apply(lambda date: date.year)
new_chicago_df.head()

,service_date,transportation_type,ridership_count,city,month,year
0,2001-01-01,bus,297192,Chicago,January,2001
1,2001-01-02,bus,780827,Chicago,January,2001
2,2001-01-03,bus,824923,Chicago,January,2001
3,2001-01-04,bus,870021,Chicago,January,2001
4,2001-01-05,bus,890426,Chicago,January,2001


In [139]:
final_chicago_df = new_chicago_df.drop(columns=['service_date'])
final_chicago_df = final_chicago_df.groupby(['year', 'month', 'transportation_type', 'city'])['ridership_count'].sum().reset_index()
final_chicago_df

,year,month,transportation_type,city,ridership_count
0,2001,April,bus,Chicago,25053791
1,2001,April,metro,Chicago,14713336
2,2001,August,bus,Chicago,24647188
3,2001,August,metro,Chicago,16014736
4,2001,December,bus,Chicago,23428582
...,...,...,...,...,...
543,2023,May,metro,Chicago,10157092
544,2023,October,bus,Chicago,15242489
545,2023,October,metro,Chicago,11509844
546,2023,September,bus,Chicago,14796427


In [130]:
final_chicago_df['transportation_type'].value_counts()

year  month      transportation_type
2001  April      bus                    30
                 metro                  30
      August     metro                  31
                 bus                    31
      December   metro                  31
                                        ..
2023  May        metro                  31
      October    bus                    31
                 metro                  31
      September  bus                    30
                 metro                  30
Name: count, Length: 548, dtype: int64

In [69]:
##### NEW YORK

In [96]:
newyork_df.head()

,Month,Agency,Ridership
0,2008-01-01,MNR,6453734
1,2008-02-01,MNR,6144377
2,2008-03-01,MNR,6602280
3,2008-04-01,MNR,6787480
4,2008-04-01,B&T,24900328


In [97]:
newyork_df['city'] = 'New York'
newyork_df.head()

,Month,Agency,Ridership,city
0,2008-01-01,MNR,6453734,New York
1,2008-02-01,MNR,6144377,New York
2,2008-03-01,MNR,6602280,New York
3,2008-04-01,MNR,6787480,New York
4,2008-04-01,B&T,24900328,New York


In [98]:
newyork_df['month'] = newyork_df['Month'].apply(find_month)
newyork_df.head()

,Month,Agency,Ridership,city,month
0,2008-01-01,MNR,6453734,New York,January
1,2008-02-01,MNR,6144377,New York,February
2,2008-03-01,MNR,6602280,New York,March
3,2008-04-01,MNR,6787480,New York,April
4,2008-04-01,B&T,24900328,New York,April


In [99]:
newyork_df['year'] = newyork_df['Month'].apply(lambda date: date.year)
newyork_df.head()

,Month,Agency,Ridership,city,month,year
0,2008-01-01,MNR,6453734,New York,January,2008
1,2008-02-01,MNR,6144377,New York,February,2008
2,2008-03-01,MNR,6602280,New York,March,2008
3,2008-04-01,MNR,6787480,New York,April,2008
4,2008-04-01,B&T,24900328,New York,April,2008


In [100]:
newyork_df = newyork_df.rename(columns={"Ridership": "ridership_count"})

In [101]:
newyork_df['Agency'].value_counts()

Agency
B&T       190
MNR       180
Bus        73
Subway     73
AAR        61
LIRR       58
SIR        13
Name: count, dtype: int64

In [102]:
def transport_type(agency):
    if agency == "MNR" or agency == "Subway" or agency == "LIRR" or agency == 'SIR':
        return "metro"
    else:
        return "bus"

newyork_df = newyork_df[newyork_df['Agency'] != "B&T"]
newyork_df['Agency'].value_counts()

Agency
MNR       180
Bus        73
Subway     73
AAR        61
LIRR       58
SIR        13
Name: count, dtype: int64

In [103]:
newyork_df['transportation_type'] = newyork_df['Agency'].apply(transport_type)

In [113]:
display(newyork_df)

,Month,Agency,ridership_count,city,month,year,transportation_type
0,2008-01-01,MNR,6453734,New York,January,2008,metro
1,2008-02-01,MNR,6144377,New York,February,2008,metro
2,2008-03-01,MNR,6602280,New York,March,2008,metro
3,2008-04-01,MNR,6787480,New York,April,2008,metro
5,2008-05-01,MNR,6840901,New York,May,2008,metro
...,...,...,...,...,...,...,...
642,2024-01-01,Subway,91419551,New York,January,2024,metro
643,2024-01-01,LIRR,5490105,New York,January,2024,metro
644,2024-01-01,AAR,928328,New York,January,2024,bus
645,2024-01-01,SIR,189970,New York,January,2024,metro


In [125]:
final_newyork_df = newyork_df.groupby(['transportation_type', 'year', 'month', 'city'])['ridership_count'].sum().reset_index()
display(final_newyork_df)

,transportation_type,year,month,city,ridership_count
0,bus,2018,April,New York,48544481
1,bus,2018,August,New York,46810522
2,bus,2018,December,New York,44033258
3,bus,2018,February,New York,44719811
4,bus,2018,January,New York,45398837
...,...,...,...,...,...
261,metro,2023,May,New York,110376738
262,metro,2023,November,New York,104558133
263,metro,2023,October,New York,110197921
264,metro,2023,September,New York,101009689


In [140]:
#test_final_ny_df = final_newyork_df.copy()

combined_df = pd.concat([final_chicago_df, final_newyork_df])
display(combined_df)

,year,month,transportation_type,city,ridership_count
0,2001,April,bus,Chicago,25053791
1,2001,April,metro,Chicago,14713336
2,2001,August,bus,Chicago,24647188
3,2001,August,metro,Chicago,16014736
4,2001,December,bus,Chicago,23428582
...,...,...,...,...,...
261,2023,May,metro,New York,110376738
262,2023,November,metro,New York,104558133
263,2023,October,metro,New York,110197921
264,2023,September,metro,New York,101009689


#### Use of postgres connector in Python to load monthly_totals table

In [141]:
for index, row in combined_df.iterrows():
    values = tuple(row)
    query = f"insert into monthly_totals (year, month, transportation_type, city, ridership_count) values {values}"
    conn.execute(query)


AttributeError: 'Connection' object has no attribute 'commit'

In [144]:
sql = """
CREATE TABLE bus_monthly_totals AS
SELECT * FROM monthly_totals
WHERE transportation_type = 'bus';
"""

conn.execute(text(sql))

In [145]:
sql = """
CREATE TABLE metro_monthly_totals AS
SELECT * FROM monthly_totals
WHERE transportation_type = 'metro';
"""

conn.execute(text(sql))